# Testing

Testing is extremely important. Without testing, you cannot be sure that your code is doing what you think. Testing is an integral part of software development, and should be done *while* you are writing code, not after the code has been written.

No doubt so far, you have been manually checking that your code does the right thing. Perhaps you are tunning your code over a particular input file and making sure that you get a correct-looking plot out at the end. This is a start but how can you be sure that there's not a subtle bug that means that the output is incorrect? And if there *is* a problem, how will you be able to work out exactly which line of code it causing it?

In order to be confident that our code it giving a correct output, a *test suite* is useful which provides a set of known inputs and checks that the code matches a set of known, expected outputs. To make it easier to locate where a bug is occuring, it's a good idea to make each individual test run over as small an amount of code as possible so that if *that* test fails, you know where to look for the problem. In Python this "small unit of code" is usually a function.

Let's get started by making sure that our `add_arrays` function matches the outputs we expect. As a reminder, this is what the file `arrays.py` looks like (though you will have a second function, `subtract_arrays` in yours):

In [1]:
%%writefile arrays.py

"""
This module contains functions for manipulating and combining Python lists.
"""

def add_arrays(x, y):
    """
    This function adds together each element of the two passed lists.

    Args:
        x (list): The first list to add
        y (list): The second list to add

    Returns:
        list: the pairwise sums of ``x`` and ``y``.

    Examples:
        >>> add_arrays([1, 4, 5], [4, 3, 5])
        [5, 7, 10]
    """
    z = []
    for x_, y_ in zip(x, y):
        z.append(x_ + y_)

    return z

Overwriting arrays.py


Since the name of the module we want to test is `arrays`, let's make a file called `test_arrays.py` which contains the following:

In [2]:
%%writefile test_arrays.py

from arrays import add_arrays

def test_add_arrays():
    a = [1, 2, 3]
    b = [4, 5, 6]
    expect = [5, 7, 9]
    
    output = add_arrays(a, b)
    
    if output == expect:
        print("OK")
    else:
        print("BROKEN")

test_add_arrays()

Overwriting test_arrays.py


This script defines a function called `test_add_arrays` which defines some known input (`a` and `b`) and a known, matching output (`expect`). It passes them to the function `add_arrays` and compares the output to `expected`. It will either print `OK` or `BROKEN` depending on whether it's working or not. Finally, we explicitly call the test function.

In [3]:
%run test_arrays.py

OK


### Exercise

Break the test by changing either `a`, `b` or `expected` and rerun the test script. Make sure that it prints `BROKEN` in this case. Change it back to a working state once you've done this.

## Asserting

The method used here works and runs the code correctly but it doesn't give very useful output. If we had five test functions in our file and three of them were failing we'd see something like:

```
OK
BROKEN
OK
BROKEN
BROKEN
```

We'd then have to cross-check back to our code to see which tests the `BROKEN`s referred to.

To be able to automatically relate the output of the failing test to the place where your test failed, you can use an `assert` statement.

An `assert` statement is followed by something which is either truthy of falsy. If it is truthy then nothing happens but if it is falsy then an exception is raised:

In [4]:
assert 5 == 5

In [5]:
assert 5 == 6

AssertionError: 

We can now use this `assert` statement in place of the `if`/`else` block:

In [6]:
%%writefile test_arrays.py

from arrays import add_arrays

def test_add_arrays():
    a = [1, 2, 3]
    b = [4, 5, 6]
    expect = [5, 7, 9]
    
    output = add_arrays(a, b)
    
    assert output == expect

test_add_arrays()

Overwriting test_arrays.py


Now when we run the test script we get nothing printed on success:

In [7]:
%run test_arrays.py

but on a failure we get an error printed like:

```
Traceback (most recent call last):
  File "test_arrays.py", line 13, in <module>
    test_add_arrays()
  File "test_arrays.py", line 11, in test_add_arrays
    assert output == expect
AssertionError
```

Which, like all exception messages gives us the location in the file at which the error occurred. This has the avantage that if we had many test functions being run it would tell us which one failed and on which line.

The downside of using an `assert` like this is that as soon as one test fails, the whole script will halt and you'll only be informed of that one test.

## pytest

There's a few things that we've been doing so far that could be improved. Firstly, for every test function that we writem we then have to explicitly call it at the bottom of the test script like `test_add_arrays()`. This is error-prone as we might write a test function and forget to call it and then we would miss any errors it would catch.

Secondly, we want nice, useful output from our test functions. Something better than the nothing/exception that a plain `assert` gives us. It would be nice to get a green `PASSED` for the good tests and a red `FAILED` for the bad ones alongside the name of the test in question.

Finally, we want to make sure that all tests are run even if a test early in the process fails.

Luckily, there is tool called *pytest* which can give us all of these things. It will work on our test script almost exactly as written with only one change needed.

Remove the call to `test_add_arrays()` on the last line of the file:

In [8]:
%%writefile test_arrays.py

from arrays import add_arrays

def test_add_arrays():
    a = [1, 2, 3]
    b = [4, 5, 6]
    expect = [5, 7, 9]
    
    output = add_arrays(a, b)
    
    assert output == expect

Overwriting test_arrays.py


And in the Terminal, run `pytest`:

In [9]:
!COLUMNS=60 venv/bin/pytest

=================== test session starts ====================
platform linux -- Python 3.7.3, pytest-5.2.2, py-1.8.0, pluggy-0.13.0
rootdir: /home/matt/projects/courses/software_engineering_best_practices
plugins: nbval-0.9.3
collected 10 items                                         

test_arrays.py .                                     [ 10%]
test_books.py .........                              [100%]

==================== 10 passed in 2.37s ====================


Pytest will do two stages. First it will try to locate all the test function that it can find and then it will run each of them in turn, reporting the results.

Here you can see that it's found that the file `test_arrays.py` contains a single test function. The green dot next tot he name of the file signifies the passing test. It then prints a summary at the end saying "1 passed".

The way that `pytest` works is that it looks for files which are called `test_*.py` or `*_test.py` and look inside those for functions whose names begin with `test`.

To see what it looks like when you have a failing test, let's deliberately break the test code by giving a wrong expected result:

In [10]:
%%writefile test_arrays.py

from arrays import add_arrays

def test_add_arrays():
    a = [1, 2, 3]
    b = [4, 5, 6]
    expect = [5, 7, 999]  # Changed this to break the test
    
    output = add_arrays(a, b)
    
    assert output == expect

Overwriting test_arrays.py


When we run this test with `pytest` it should tell us that the test is indeed failing:

In [11]:
!COLUMNS=60 venv/bin/pytest

=================== test session starts ====================
platform linux -- Python 3.7.3, pytest-5.2.2, py-1.8.0, pluggy-0.13.0
rootdir: /home/matt/projects/courses/software_engineering_best_practices
plugins: nbval-0.9.3
collected 10 items                                         

test_arrays.py F                                     [ 10%]
test_books.py .........                              [100%]

========================= FAILURES =========================
_____________________ test_add_arrays ______________________

    def test_add_arrays():
        a = [1, 2, 3]
        b = [4, 5, 6]
        expect = [5, 7, 999]  # Changed this to break the test
    
        output = add_arrays(a, b)
    
>       assert output == expect
E       assert [5, 7, 9] == [5, 7, 999]
E         At index 2 diff: 9 != 999
E         Use -v to get the full diff

test_arrays.py:11: AssertionError
=============== 1 failed, 9 passed in 2.51s ================


The output from this is better than we saw with the plain `assert`. It's printing the full context of the contents of the test function with the line where the `assert` is failing being marked with a `>`. It then gives an expanded explanation of why the assert failed. Before we just got `AssertionError` but now it prints out the contents of `output` and `expect` and tells us that at index 2 of the list it's finding a `9` where we told it to expect a `999`.

Before continuing, make sure that you change the file back to its previous contents by changing that `999` back to a `9`.

In [12]:
%%writefile test_arrays.py

from arrays import add_arrays

def test_add_arrays():
    a = [1, 2, 3]
    b = [4, 5, 6]
    expect = [5, 7, 9]  # Changed this back to 9
    
    output = add_arrays(a, b)
    
    assert output == expect

Overwriting test_arrays.py


### Exercise

Write a test which tests your `subtract_arrays` function from the previous chapter. Make sure it passes with a correct input/output and correctly fails if you break it on purpose. [<small>answer</small>](answer_subtract_test.ipynb)

## Avoid repeating ourselves

Having a single test for a function is already infinitely better than having none, but one test only gives you so much confidence. The real power of a test suite is being able to test your functions under lots of different conditions.

Lets add a second test to check a different set of inputs and outputs to the `add_arrays` function and check that it passes:

In [13]:
%%writefile test_arrays.py

from arrays import add_arrays

def test_add_arrays1():
    a = [1, 2, 3]
    b = [4, 5, 6]
    expect = [5, 7, 9]
    
    output = add_arrays(a, b)
    
    assert output == expect

def test_add_arrays2():
    a = [-1, -5, -3]
    b = [-4, -3, 0]
    expect = [-5, -8, -3]
    
    output = add_arrays(a, b)
    
    assert output == expect

Overwriting test_arrays.py


When we run `pytest` we can optionally pass the `-v` flag which puts it in *verbose* mode. This will print out the tests being run, one per line which I find a more useful view most of the time:

In [14]:
!COLUMNS=60 venv/bin/pytest -v

=================== test session starts ====================
platform linux -- Python 3.7.3, pytest-5.2.2, py-1.8.0, pluggy-0.13.0 -- /home/matt/projects/courses/software_engineering_best_practices/venv/bin/python3
cachedir: .pytest_cache
rootdir: /home/matt/projects/courses/software_engineering_best_practices
plugins: nbval-0.9.3
collected 11 items                                         

test_arrays.py::test_add_arrays1 PASSED              [  9%]
test_arrays.py::test_add_arrays2 PASSED              [ 18%]
test_books.py::test_word_counts[hat-33] PASSED       [ 27%]
test_books.py::test_word_counts[freedom-71] PASSED   [ 36%]
test_books.py::test_word_counts[electricity-1] PASSED [ 45%]
test_books.py::test_word_counts[testing-3] PASSED    [ 54%]
test_books.py::test_word_counts[Prince-1498] PASSED  [ 63%]
test_books.py::test_word_counts[internet-0] PASSED   [ 72%]
test_books.py::test_word_counts[Russia-71] PASSED    [ 81%]
test_books.py::test_word_counts[Pierre-1260] PASSED  [ 90%]
test_

We see both tests being run and passing. This will work well but we've had to repeat ourselves almost entirely in each test function. The only difference between the two functions is the inputs and outputs under test. Usually in this case in a function you would take these things as arguments and we can do the same thing here.

The actual logic of the function is the following:

```python
def test_add_arrays(a, b, expect):
    output = add_arrays(a, b)
    assert output == expect
```

We then just need a way of passing the data we want to check into this function. For this, pytest provides a feature called *parametrisation*. We label our function with a *decoration* which allows pytest to run it mutliple times with different data.

To use this feature we must import the `pytest` module and use the `pytest.mark.parametrize` decorator like the following:

In [15]:
%%writefile test_arrays.py

import pytest

from arrays import add_arrays

@pytest.mark.parametrize("a, b, expect", [
    ([1, 2, 3],    [4, 5, 6],   [5, 7, 9]),
    ([-1, -5, -3], [-4, -3, 0], [-5, -8, -3]),
])
def test_add_arrays(a, b, expect):
    output = add_arrays(a, b)
    
    assert output == expect

Overwriting test_arrays.py


The `parametrize` decorator takes two arguments:
1. a string containing the names of the variables you want to pass in (`"a, b, expect"`)
2. a list containing the values of the variables you want to pass in

In this case, the test will be run twice. Once with each of the following values:
1. `a=[1, 2, 3]`, `b=[4, 5, 6]`, `expect=[5, 7, 9]`
2. `a=[-1, -5, -3]`, `b=[-4, -3, 0]`, `expect=[-5, -8, -3]`

In [16]:
!COLUMNS=60 venv/bin/pytest -v

=================== test session starts ====================
platform linux -- Python 3.7.3, pytest-5.2.2, py-1.8.0, pluggy-0.13.0 -- /home/matt/projects/courses/software_engineering_best_practices/venv/bin/python3
cachedir: .pytest_cache
rootdir: /home/matt/projects/courses/software_engineering_best_practices
plugins: nbval-0.9.3
collected 11 items                                         

test_arrays.py::test_add_arrays[a0-b0-expect0] PASSED [  9%]
test_arrays.py::test_add_arrays[a1-b1-expect1] PASSED [ 18%]
test_books.py::test_word_counts[hat-33] PASSED       [ 27%]
test_books.py::test_word_counts[freedom-71] PASSED   [ 36%]
test_books.py::test_word_counts[electricity-1] PASSED [ 45%]
test_books.py::test_word_counts[testing-3] PASSED    [ 54%]
test_books.py::test_word_counts[Prince-1498] PASSED  [ 63%]
test_books.py::test_word_counts[internet-0] PASSED   [ 72%]
test_books.py::test_word_counts[Russia-71] PASSED    [ 81%]
test_books.py::test_word_counts[Pierre-1260] PASSED  [ 90%]
tes

### Exercise

- Add some more parameters to the `test_add_arrays` function.
- Parametrise the `subtract_arrays` test function.

## Failing correctly

The interface of a function is made up of the *parameters* it expects and the values that it *returns*. If a user of a function knows these things then they are able to use it correctly. This is why we make sure to include this information in the docstring for all our functions.

The other thing that is part of the interface of a function is any exceptions that are *raised* by it. If you need a refresher on exceptionns and error handling in Python, take a look at [the chapter on it in the Intermediate Python course](https://milliams.gitlab.io/intermediate_python/05%20Exceptions.html).

To add explicit error handling to our function we need to do two things:
1. add in a conditional `raise` statement:
   ```python
   if len(x) != len(y):
       raise ValueError("Both arrays must have the same length.")
   ```
2. document in the docstring the fact that the function may raise something:
   ```
   Raises:
       ValueError: If the length of the lists ``x`` and ``y`` are different.
   ```

Let's add these to `arrays.py`:

In [17]:
%%writefile arrays.py

"""
This module contains functions for manipulating and combining Python lists.
"""

def add_arrays(x, y):
    """
    This function adds together each element of the two passed lists.

    Args:
        x (list): The first list to add
        y (list): The second list to add

    Returns:
        list: the pairwise sums of ``x`` and ``y``.
    
    Raises:
        ValueError: If the length of the lists ``x`` and ``y`` are different.

    Examples:
        >>> add_arrays([1, 4, 5], [4, 3, 5])
        [5, 7, 10]
    """
    
    if len(x) != len(y):
        raise ValueError("Both arrays must have the same length.")
    
    z = []
    for x_, y_ in zip(x, y):
        z.append(x_ + y_)

    return z

Overwriting arrays.py


In [18]:
%%writefile arrays.py

"""
This module contains functions for manipulating and combining Python lists.
"""

def add_arrays(x, y):
    """
    This function adds together each element of the two passed lists.

    Args:
        x (list): The first list to add
        y (list): The second list to add

    Returns:
        list: the pairwise sums of ``x`` and ``y``.

    Examples:
        >>> add_arrays([1, 4, 5], [4, 3, 5])
        [5, 7, 10]
    """

    if len(x) != len(y):
        raise ValueError("Both arrays must have the same length.")

    z = []
    for x_, y_ in zip(x, y):
        z.append(x_ + y_)

    return z

def subtract_arrays(x, y):
    """
    This function subtracts from each other each element of the two passed lists.

    Args:
        x (list): The first list
        y (list): The second list

    Returns:
        list: the pairwise difference of ``x`` and ``y``.

    Examples:
        >>> subtract_arrays([1, 4, 5], [4, 3, 5])
        [-3, 1, 0]
    """

    if len(x) != len(y):
        raise ValueError("Both arrays must have the same length.")

    z = []
    for x_, y_ in zip(x, y):
        z.append(x_ - y_)

    return z

Overwriting arrays.py


We can then test that the function correctly raises the exception when passes appropriate data.  Inside a pytest function we can require that a specific exception is raised by using [`pytest.raises`](https://docs.pytest.org/en/latest/reference.html#pytest-raises) in a `with` block. `pytest.raises` takes as an argument the type of an exception and if the block ends without that exception habing been rasied, will fail the test.

It may seem strange that we're testing-for and *requiring* that the function raises an error but it's important that if we've told our users that the code will produce a certain error in specific circumstances that it does indeed do as we promise.

In our code we add a new test called `test_add_arrays_error` which does the check we require:

In [19]:
%%writefile test_arrays.py

import pytest

from arrays import add_arrays

@pytest.mark.parametrize("a, b, expect", [
    ([1, 2, 3],    [4, 5, 6],   [5, 7, 9]),
    ([-1, -5, -3], [-4, -3, 0], [-5, -8, -3]),
])
def test_add_arrays(a, b, expect):
    output = add_arrays(a, b)
    
    assert output == expect

def test_add_arrays_error():
    a = [1, 2, 3]
    b = [4, 5]
    with pytest.raises(ValueError):
        output = add_arrays(a, b)

Overwriting test_arrays.py


In [20]:
!COLUMNS=60 venv/bin/pytest -v

=================== test session starts ====================
platform linux -- Python 3.7.3, pytest-5.2.2, py-1.8.0, pluggy-0.13.0 -- /home/matt/projects/courses/software_engineering_best_practices/venv/bin/python3
cachedir: .pytest_cache
rootdir: /home/matt/projects/courses/software_engineering_best_practices
plugins: nbval-0.9.3
collected 12 items                                         

test_arrays.py::test_add_arrays[a0-b0-expect0] PASSED [  8%]
test_arrays.py::test_add_arrays[a1-b1-expect1] PASSED [ 16%]
test_arrays.py::test_add_arrays_error PASSED         [ 25%]
test_books.py::test_word_counts[hat-33] PASSED       [ 33%]
test_books.py::test_word_counts[freedom-71] PASSED   [ 41%]
test_books.py::test_word_counts[electricity-1] PASSED [ 50%]
test_books.py::test_word_counts[testing-3] PASSED    [ 58%]
test_books.py::test_word_counts[Prince-1498] PASSED  [ 66%]
test_books.py::test_word_counts[internet-0] PASSED   [ 75%]
test_books.py::test_word_counts[Russia-71] PASSED    [ 83%]
tes

### Exercise

- Add a runtime test to the `subtract_arrays` function to check for unequal-length arguments.
- Add a test for the exception.

## Doctests

If you remember from when we were documenting out `add_arrays` function, we had a small section which gave the reader an example of how to use the function:

```
Examples:
    >>> add_arrays([1, 4, 5], [4, 3, 5])
    [5, 7, 10]
```

Since this is valid Python code, we can ask pytest to run this code and check that the output we claimed would be returned is correct. If we pass `--doctest-modules` to the `pytest` command, it will search `.py` files for docstrings with example blocks and run them:

In [21]:
!COLUMNS=60 venv/bin/pytest -v --doctest-modules

=================== test session starts ====================
platform linux -- Python 3.7.3, pytest-5.2.2, py-1.8.0, pluggy-0.13.0 -- /home/matt/projects/courses/software_engineering_best_practices/venv/bin/python3
cachedir: .pytest_cache
rootdir: /home/matt/projects/courses/software_engineering_best_practices
plugins: nbval-0.9.3
collected 14 items                                         

arrays.py::arrays.add_arrays PASSED                  [  7%]
arrays.py::arrays.subtract_arrays PASSED             [ 14%]
test_arrays.py::test_add_arrays[a0-b0-expect0] PASSED [ 21%]
test_arrays.py::test_add_arrays[a1-b1-expect1] PASSED [ 28%]
test_arrays.py::test_add_arrays_error PASSED         [ 35%]
test_books.py::test_word_counts[hat-33] PASSED       [ 42%]
test_books.py::test_word_counts[freedom-71] PASSED   [ 50%]
test_books.py::test_word_counts[electricity-1] PASSED [ 57%]
test_books.py::test_word_counts[testing-3] PASSED    [ 64%]
test_books.py::test_word_counts[Prince-1498] PASSED  [ 71%]
tes

We see here the `arrays.py::arrays.add_arrays` test which has passed. Ignore the warning about deprecation, this is from a third-party module which is leaking through.

Doctests are a really valuable thing to have in your test suite as they ensure that any examples that you are giving work as expected. It's not uncommon for the code to change and for the documentation to be left behind and begin able to automatically check all your examples avoids this.

### Exercise

See what happens when you break your doctest and run `pytest` again.

## Running specific tests

As you increase the number of tests you will come across situations where you only want to run a particular test. TO do this, you follow pass the name of the test, as printed by `pytest -v` as an argument to `pytest`. So, if we want to run all tests in `test_arrays.py` we do:

In [22]:
!COLUMNS=60 venv/bin/pytest -v test_arrays.py

=================== test session starts ====================
platform linux -- Python 3.7.3, pytest-5.2.2, py-1.8.0, pluggy-0.13.0 -- /home/matt/projects/courses/software_engineering_best_practices/venv/bin/python3
cachedir: .pytest_cache
rootdir: /home/matt/projects/courses/software_engineering_best_practices
plugins: nbval-0.9.3
collected 3 items                                          

test_arrays.py::test_add_arrays[a0-b0-expect0] PASSED [ 33%]
test_arrays.py::test_add_arrays[a1-b1-expect1] PASSED [ 66%]
test_arrays.py::test_add_arrays_error PASSED         [100%]

==================== 3 passed in 0.01s =====================


Or, if we want to specifically run the `test_add_arrays` test:

In [23]:
!COLUMNS=60 venv/bin/pytest -v test_arrays.py::test_add_arrays

=================== test session starts ====================
platform linux -- Python 3.7.3, pytest-5.2.2, py-1.8.0, pluggy-0.13.0 -- /home/matt/projects/courses/software_engineering_best_practices/venv/bin/python3
cachedir: .pytest_cache
rootdir: /home/matt/projects/courses/software_engineering_best_practices
plugins: nbval-0.9.3
collected 2 items                                          

test_arrays.py::test_add_arrays[a0-b0-expect0] PASSED [ 50%]
test_arrays.py::test_add_arrays[a1-b1-expect1] PASSED [100%]

==================== 2 passed in 0.01s =====================


Or, if we want to run one test specifically:

In [24]:
!COLUMNS=60 venv/bin/pytest -v test_arrays.py::test_add_arrays[a0-b0-expect0]

=================== test session starts ====================
platform linux -- Python 3.7.3, pytest-5.2.2, py-1.8.0, pluggy-0.13.0 -- /home/matt/projects/courses/software_engineering_best_practices/venv/bin/python3
cachedir: .pytest_cache
rootdir: /home/matt/projects/courses/software_engineering_best_practices
plugins: nbval-0.9.3
collected 1 item                                           

test_arrays.py::test_add_arrays[a0-b0-expect0] PASSED [100%]

==================== 1 passed in 0.01s =====================


Take a look at the output of `pytest -h` for more options. For example, you can tell `pytest` to only run the tests that failed on the last run with `pytest --last-failed`.

## Input data for tests

As we saw above when using parametrisation, it's often useful to split your test function into two parts:
1. The data to be tested
2. The code to do the test

This is because we had a situation where we had one test function and multiple examples to test. The opposite situation also happens where we have multiple test functions, all of which want the same input data.

The name that pytest uses for "data which is provided to test functions" is *fixture* since it *fixes* a set of data against which to test.

We'll start with the example of the `add_arrays` function to demonstrate the syntax but soon we'll need to use a example which demonstates the benefits more.

To make things clearer, we'll trim down the test file back to the basics. Just one test for `add_arrays` and one for `subtract_arrays`:

In [25]:
%%writefile test_arrays.py

from arrays import add_arrays, subtract_arrays

def test_add_arrays():
    a = [1, 2, 3]
    b = [4, 5, 6]
    expect = [5, 7, 9]
    
    output = add_arrays(a, b)
    
    assert output == expect

def test_subtract_arrays():
    a = [1, 2, 3]
    b = [4, 5, 6]
    expect = [-3, -3, -3]
    
    output = subtract_arrays(a, b)
    
    assert output == expect

Overwriting test_arrays.py


Both of these tests use the same values for `a` and `b`. In this case it's not a big problem that we're repeating ourselves here but in a more complex test suite the data were testing over may be very complicated or slow to create.

To create our fixture we define a function which is decorated with the `pytest.fixture` decorator. Apart from that, all the function needs do is return the data we want to provide to our tests:

```python
import pytest

@pytest.fixture
def pair_of_lists():
    return [1, 2, 3], [4, 5, 6]
```

To make the test functions make use of the fixture, we use the name of the fixture (`pair_of_lists`) as a parameter of the test function, similar to how we did with parametrisation:

```python
def test_add_arrays(pair_of_lists):
    ...
```

The data is now available inside the function using that name and we can use it however we wish:

```python
def test_add_arrays(pair_of_lists):
    a, b = pair_of_lists
    ...
```

In [26]:
%%writefile test_arrays.py

import pytest

from arrays import add_arrays, subtract_arrays

@pytest.fixture
def pair_of_lists():
    return [1, 2, 3], [4, 5, 6]

def test_add_arrays(pair_of_lists):
    a, b = pair_of_lists
    expect = [5, 7, 9]
    
    output = add_arrays(a, b)
    
    assert output == expect

def test_subtract_arrays(pair_of_lists):
    a, b = pair_of_lists
    expect = [-3, -3, -3]
    
    output = subtract_arrays(a, b)
    
    assert output == expect

Overwriting test_arrays.py


When we run the test suite, pytest will automatically run the `pair_of_lists` function for each test and pass in the result.

In [27]:
!COLUMNS=60 venv/bin/pytest -v

=================== test session starts ====================
platform linux -- Python 3.7.3, pytest-5.2.2, py-1.8.0, pluggy-0.13.0 -- /home/matt/projects/courses/software_engineering_best_practices/venv/bin/python3
cachedir: .pytest_cache
rootdir: /home/matt/projects/courses/software_engineering_best_practices
plugins: nbval-0.9.3
collected 11 items                                         

test_arrays.py::test_add_arrays PASSED               [  9%]
test_arrays.py::test_subtract_arrays PASSED          [ 18%]
test_books.py::test_word_counts[hat-33] PASSED       [ 27%]
test_books.py::test_word_counts[freedom-71] PASSED   [ 36%]
test_books.py::test_word_counts[electricity-1] PASSED [ 45%]
test_books.py::test_word_counts[testing-3] PASSED    [ 54%]
test_books.py::test_word_counts[Prince-1498] PASSED  [ 63%]
test_books.py::test_word_counts[internet-0] PASSED   [ 72%]
test_books.py::test_word_counts[Russia-71] PASSED    [ 81%]
test_books.py::test_word_counts[Pierre-1260] PASSED  [ 90%]
test_

It might be hard to see the benefit of fixtures with this rather contrived example so lets take a look at a more sensible one where using a fixture makes sense.

Make a new file called `test_books.py` which contains the following:

In [28]:
%%writefile books.py

def word_count(text: str, word: str='') -> int:
    """
    Count the number of occurences of ``word`` in a string.
    If ``word`` is not set, count all words.
    """
    if word:
        count = 0
        for text_word in text.split():
            if text_word == word:
                count += 1
        return count
    else:
        return len(text.split())

Overwriting books.py


To test this function we want a corpus of text to test it over. For the purposes of this example and to simulate a complex data input, we will download the contents of a long book from Project Gutenberg. Our test function uses [`urllib.request`](https://docs.python.org/3/library/urllib.request.html) to download the text, converts it to a string and passes that to the `word_count` function. At first we will simply check that the word "hat" appears 33 times in the book:

In [29]:
%%writefile test_books.py

import urllib.request

from books import word_count

def test_word_counts():
    url = "https://www.gutenberg.org/files/2600/2600-0.txt"
    book_text = urllib.request.urlopen(url).read().decode('utf-8')
    assert word_count(book_text, "hat") == 33

Overwriting test_books.py


In [30]:
!COLUMNS=60 venv/bin/pytest -v test_books.py

=================== test session starts ====================
platform linux -- Python 3.7.3, pytest-5.2.2, py-1.8.0, pluggy-0.13.0 -- /home/matt/projects/courses/software_engineering_best_practices/venv/bin/python3
cachedir: .pytest_cache
rootdir: /home/matt/projects/courses/software_engineering_best_practices
plugins: nbval-0.9.3
collected 1 item                                           

test_books.py::test_word_counts PASSED               [100%]

==================== 1 passed in 1.91s =====================


...took ~2 seconds...

...not too bad, but if we want to test multiple...

In [31]:
%%writefile test_books.py

import urllib.request

import pytest

from books import word_count

@pytest.mark.parametrize('word,count',  [
    ('hat', 33),
    ('freedom', 71),
    ('electricity', 1),
    ('testing', 3),
    ('Prince', 1498),
    ('internet', 0),
    ('Russia', 71),
    ('Pierre', 1260),
    (None, 566311),
])
def test_word_counts(word, count):
    url = "https://www.gutenberg.org/files/2600/2600-0.txt"
    book_text = urllib.request.urlopen(url).read().decode('utf-8')
    assert word_count(book_text, word) == count

Overwriting test_books.py


In [32]:
!COLUMNS=60 venv/bin/pytest -v test_books.py

=================== test session starts ====================
platform linux -- Python 3.7.3, pytest-5.2.2, py-1.8.0, pluggy-0.13.0 -- /home/matt/projects/courses/software_engineering_best_practices/venv/bin/python3
cachedir: .pytest_cache
rootdir: /home/matt/projects/courses/software_engineering_best_practices
plugins: nbval-0.9.3
collected 9 items                                          

test_books.py::test_word_counts[hat-33] PASSED       [ 11%]
test_books.py::test_word_counts[freedom-71] PASSED   [ 22%]
test_books.py::test_word_counts[electricity-1] PASSED [ 33%]
test_books.py::test_word_counts[testing-3] PASSED    [ 44%]
test_books.py::test_word_counts[Prince-1498] PASSED  [ 55%]
test_books.py::test_word_counts[internet-0] PASSED   [ 66%]
test_books.py::test_word_counts[Russia-71] PASSED    [ 77%]
test_books.py::test_word_counts[Pierre-1260] PASSED  [ 88%]
test_books.py::test_word_counts[None-566311] PASSED  [100%]

==================== 9 passed in 15.91s ====================


...It took nine times as long...

...move the slow setup into a fixture...

In [33]:
%%writefile test_books.py

import urllib.request

import pytest

from books import word_count

@pytest.fixture()
def long_book():
    url = "https://www.gutenberg.org/files/2600/2600-0.txt"
    book_text = urllib.request.urlopen(url).read().decode('utf-8')
    return book_text

@pytest.mark.parametrize('word,count',  [
    ('hat', 33),
    ('freedom', 71),
    ('electricity', 1),
    ('testing', 3),
    ('Prince', 1498),
    ('internet', 0),
    ('Russia', 71),
    ('Pierre', 1260),
    (None, 566311),
])
def test_word_counts(long_book, word, count):
    assert word_count(long_book, word) == count

Overwriting test_books.py


In [34]:
!COLUMNS=60 venv/bin/pytest -v test_books.py

=================== test session starts ====================
platform linux -- Python 3.7.3, pytest-5.2.2, py-1.8.0, pluggy-0.13.0 -- /home/matt/projects/courses/software_engineering_best_practices/venv/bin/python3
cachedir: .pytest_cache
rootdir: /home/matt/projects/courses/software_engineering_best_practices
plugins: nbval-0.9.3
collected 9 items                                          

test_books.py::test_word_counts[hat-33] PASSED       [ 11%]
test_books.py::test_word_counts[freedom-71] PASSED   [ 22%]
test_books.py::test_word_counts[electricity-1] PASSED [ 33%]
test_books.py::test_word_counts[testing-3] PASSED    [ 44%]
test_books.py::test_word_counts[Prince-1498] PASSED  [ 55%]
test_books.py::test_word_counts[internet-0] PASSED   [ 66%]
test_books.py::test_word_counts[Russia-71] PASSED    [ 77%]
test_books.py::test_word_counts[Pierre-1260] PASSED  [ 88%]
test_books.py::test_word_counts[None-566311] PASSED  [100%]

==================== 9 passed in 17.90s ====================


...need to use `scope="module"` as an argument to `pytest.fixture`...

In [35]:
%%writefile test_books.py

import urllib.request

import pytest

from books import word_count

@pytest.fixture(scope="module")
def long_book():
    url = "https://www.gutenberg.org/files/2600/2600-0.txt"
    book_text = urllib.request.urlopen(url).read().decode('utf-8')
    return book_text

@pytest.mark.parametrize('word,count',  [
    ('hat', 33),
    ('freedom', 71),
    ('electricity', 1),
    ('testing', 3),
    ('Prince', 1498),
    ('internet', 0),
    ('Russia', 71),
    ('Pierre', 1260),
    (None, 566311),
])
def test_word_counts(long_book, word, count):
    assert word_count(long_book, word) == count

Overwriting test_books.py


In [36]:
!COLUMNS=60 venv/bin/pytest -v test_books.py

=================== test session starts ====================
platform linux -- Python 3.7.3, pytest-5.2.2, py-1.8.0, pluggy-0.13.0 -- /home/matt/projects/courses/software_engineering_best_practices/venv/bin/python3
cachedir: .pytest_cache
rootdir: /home/matt/projects/courses/software_engineering_best_practices
plugins: nbval-0.9.3
collected 9 items                                          

test_books.py::test_word_counts[hat-33] PASSED       [ 11%]
test_books.py::test_word_counts[freedom-71] PASSED   [ 22%]
test_books.py::test_word_counts[electricity-1] PASSED [ 33%]
test_books.py::test_word_counts[testing-3] PASSED    [ 44%]
test_books.py::test_word_counts[Prince-1498] PASSED  [ 55%]
test_books.py::test_word_counts[internet-0] PASSED   [ 66%]
test_books.py::test_word_counts[Russia-71] PASSED    [ 77%]
test_books.py::test_word_counts[Pierre-1260] PASSED  [ 88%]
test_books.py::test_word_counts[None-566311] PASSED  [100%]

==================== 9 passed in 2.55s =====================


...now fast...

### Exercise

Add some more parameters to the test and check that it doesn't take any longer to run

What to commit

... CI ...